In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

2025-06-02 17:59:07.857220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748887148.045919      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748887148.101147      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_images=np.load('/kaggle/input/poc-dataset-image-caption/nano_processed_train_images.npy')
train_input_captions=np.load('/kaggle/input/poc-dataset-image-caption/processed_train_input_captions.npy')
train_label_captions=np.load('/kaggle/input/poc-dataset-image-caption/processed_train_label_captions.npy')

In [3]:
train_images.shape

(5, 49, 2048)

In [4]:
print(train_input_captions.shape,train_label_captions.shape)

(28320, 37) (28320, 37)


In [5]:
train_input_captions=train_input_captions[0:5,:]
train_label_captions=train_label_captions[0:5,:]

In [6]:
print(train_input_captions.shape,train_label_captions.shape)

(5, 37) (5, 37)


In [7]:
train_input_captions[0]

array([     2,  43013,  98973, 188484,  43013, 285071, 129288, 192976,
       102812, 373320,  43013, 325902, 268049, 341258, 188484,  54276,
       138044, 384125,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0])

In [8]:
train_label_captions[0]

array([ 43013,  98973, 188484,  43013, 285071, 129288, 192976, 102812,
       373320,  43013, 325902, 268049, 341258, 188484,  54276, 138044,
       384125,      3,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0])

In [9]:
dataset=tf.data.Dataset.from_tensor_slices(((np.expand_dims(train_images[0],axis=0),np.expand_dims(train_input_captions[0],axis=0)),np.expand_dims(train_label_captions[0],axis=0))).batch(1)

I0000 00:00:1748887230.524839      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1748887230.525700      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [48]:
dataset=tf.data.Dataset.from_tensor_slices(((train_images,train_input_captions),train_label_captions)).batch(1)

In [10]:
emb_matrix=np.load('/kaggle/input/poc-dataset-image-caption/embedding_matrix.npy')

In [13]:
def model_build():
    image=tf.keras.Input((49,2048))
    input_caption=tf.keras.Input((37,))

    x_image=Dense(units=1024,activation='relu')(image)
    x_image=Dense(units=512,activation='relu')(x_image) #key,value

    embedding_layer =Embedding(400004,300,trainable=False,mask_zero=True)

    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])

    x_caption=embedding_layer(input_caption)

    x_caption=LSTM(units=512,activation='relu',return_sequences=True)(x_caption) #query

    attention = MultiHeadAttention(num_heads=1,key_dim=512)(query=x_caption,value=x_image)

    x=tf.keras.layers.Add()([x_caption,attention])
    x=LayerNormalization()(x)

    x=LSTM(units=256,activation='relu',return_sequences=True)(x)
    output=Dense(units=400004,activation='softmax')(x)

    model=tf.keras.Model(inputs=[image,input_caption],outputs=output)
    return model

In [49]:
model=model_build()

In [50]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 37)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_6             │ (None, 49, 2048)       │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 37, 300)        │    120,001,200 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_3 (NotEqual)    │ (None, 37)             │              0 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 49, 1024)       │      2,098,176 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_5 (LSTM)             │ (None, 37, 512)        │      1,665,024 │ embedding_3[0][0],     │
│                           │                        │                │ not_equal_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 49, 512)        │        524,800 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 37, 512)        │      1,050,624 │ lstm_5[0][0],          │
│ (MultiHeadAttention)      │                        │                │ not_equal_3[0][0],     │
│                           │                        │                │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 37, 512)        │              0 │ lstm_5[0][0],          │
│                           │                        │                │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 37, 512)        │          1,024 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ logical_or_2 (LogicalOr)  │ (None, 37)             │              0 │ not_equal_3[0][0],     │
│                           │                        │                │ not_equal_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ (None, 37, 256)        │        787,456 │ layer_normalization_2… │
│                           │                        │                │ logical_or_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 37, 400004)     │    102,801,028 │ lstm_6[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 228,929,332 (873.30 MB)

 Trainable params: 108,928,132 (415.53 MB)

 Non-trainable params: 120,001,200 (457.77 MB)

In [37]:
def masked_loss(y_true, y_pred):
    # y_true: (batch_size, seq_len)
    # y_pred: (batch_size, seq_len, vocab_size)
    y_true = tf.cast(y_true, tf.int32)

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
    loss = loss_fn(y_true, y_pred)  # (batch_size, seq_len)
    
    # Create mask for non-padding tokens
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    
    # Apply mask
    loss *= mask

    # Return average loss per batch
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

def masked_accuracy(y_true, y_pred):
    # y_true: (batch_size, seq_len)
    # y_pred: (batch_size, seq_len, vocab_size)
    y_true = tf.cast(y_true, tf.int32)

    y_pred_classes = tf.argmax(y_pred, axis=-1, output_type=tf.int32)  # (batch_size, seq_len)
    matches = tf.cast(tf.equal(y_true, y_pred_classes), tf.float32)

    # Mask out padding tokens
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    matches *= mask

    return tf.reduce_sum(matches) / tf.reduce_sum(mask)


In [62]:
model.compile(optimizer=Adam(learning_rate=0.01),loss=masked_loss,metrics=[masked_accuracy])

In [52]:
history=model.fit(dataset,epochs=100)

Epoch 1/100


W0000 00:00:1748889567.070043      96 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 56ms/step - loss: 12.4871 - masked_accuracy: 0.0575
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 7.6388 - masked_accuracy: 0.1883
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 6.3010 - masked_accuracy: 0.0954
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 4.4692 - masked_accuracy: 0.1372
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 4.2151 - masked_accuracy: 0.1044
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 5.9252 - masked_accuracy: 0.1081
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 6.4402 - masked_accuracy: 0.1761
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 8.6035 - masked_accuracy: 0.0735  
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 8.9203 - masked_accuracy: 0.0680
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 6.0275 - masked_accuracy: 0.1165
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 5.1069 - masked_accuracy: 0.1883
Epoch

In [53]:
history2=model.fit(dataset,epochs=150,initial_epoch=history.epoch[-1])

Epoch 100/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.7465 - masked_accuracy: 0.6553
Epoch 101/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.7214 - masked_accuracy: 0.6783
Epoch 102/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.7159 - masked_accuracy: 0.7013
Epoch 103/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.7072 - masked_accuracy: 0.7352
Epoch 104/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.6981 - masked_accuracy: 0.7582
Epoch 105/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.6675 - masked_accuracy: 0.7352
Epoch 106/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.6611 - masked_accuracy: 0.7352
Epoch 107/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.6771 - masked_accuracy: 0.7243
Epoch 108/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.8886 - masked_accuracy: 0.6662
Epoch 109/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 1.4683 - masked_accuracy: 0.5341
Epoch 110/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 1.0637 - m

In [54]:
history3=model.fit(dataset,epochs=200,initial_epoch=history2.epoch[-1])

Epoch 150/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.5998 - masked_accuracy: 0.7970
Epoch 151/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.5050 - masked_accuracy: 0.7970
Epoch 152/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.4033 - masked_accuracy: 0.8612
Epoch 153/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.3569 - masked_accuracy: 0.8612
Epoch 154/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.3463 - masked_accuracy: 0.8382
Epoch 155/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.4545 - masked_accuracy: 0.8612
Epoch 156/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.3867 - masked_accuracy: 0.8842
Epoch 157/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.3370 - masked_accuracy: 0.8612
Epoch 158/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.2761 - masked_accuracy: 0.8842
Epoch 159/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.2771 - masked_accuracy: 0.8842
Epoch 160/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.2587 - m

In [63]:
history4=model.fit(dataset,epochs=220,initial_epoch=history3.epoch[-1])

Epoch 200/220


W0000 00:00:1748889923.361732      97 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - loss: 7.0113 - masked_accuracy: 0.3794
Epoch 201/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 11.4600 - masked_accuracy: 0.2173
Epoch 202/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 7.9612 - masked_accuracy: 0.1967
Epoch 203/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 4.0305 - masked_accuracy: 0.1194
Epoch 204/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 2.7832 - masked_accuracy: 0.1864
Epoch 205/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 2.3703 - masked_accuracy: 0.3278
Epoch 206/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 2.3335 - masked_accuracy: 0.3264
Epoch 207/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 2.1915 - masked_accuracy: 0.3241
Epoch 208/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 2.2130 - masked_accuracy: 0.2987
Epoch 209/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 2.0634 - masked_accuracy: 0.3296
Epoch 210/220
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 2.0567 - masked_accura

In [42]:
out=model.predict(dataset)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [43]:
out.shape

(1, 37, 400004)

In [44]:
out=np.argmax(out,-1)

In [45]:
out.shape

(1, 37)

In [46]:
out

array([[ 43013,  98973, 188484,  43013, 285071, 129288, 192976, 102812,
        373320,  43013, 325902, 268049, 341258, 188484,  54276, 138044,
        384125,      3,      3,      3,      3,      3,      3,      3,
             3,      3,      3,      3,      3,      3,      3,      3,
             3,      3,      3,      3,      3]])

In [47]:
train_label_captions[0]

array([ 43013,  98973, 188484,  43013, 285071, 129288, 192976, 102812,
       373320,  43013, 325902, 268049, 341258, 188484,  54276, 138044,
       384125,      3,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0])